<a href="https://colab.research.google.com/github/shafayat1004/Parallel-Programming-Assignments/blob/main/MP0/MP0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jul 23 01:31:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ma2fwoks
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ma2fwoks
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=82d2528d4e0bc4d12116bb3a9baff2d903e8308a21978e66906398cfa8afd84b
  Stored in directory: /tmp/pip-ephem-wheel-cache-dfbqq8d4/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%wget -p https://raw.githubusercontent.com/shafayat1004/Parallel-Programming-Assignments/main/include/wb.h

Writing wb.h


In [ ]:
%%writefile test.cu
#include "wb.h"

//@@ The purpose of this code is to become familiar with the submission
//@@ process. Do not worry if you do not understand all the details of
//@@ the code.

int main(int argc, char **argv) {
  int deviceCount;

  wbArg_read(argc, argv);

  cudaGetDeviceCount(&deviceCount);

  wbTime_start(GPU, "Getting GPU Data."); //@@ start a timer

  for (int dev = 0; dev < deviceCount; dev++) {
    cudaDeviceProp deviceProp;

    cudaGetDeviceProperties(&deviceProp, dev);

    if (dev == 0) {
      if (deviceProp.major == 9999 && deviceProp.minor == 9999) {
        wbLog(TRACE, "No CUDA GPU has been detected");
        return -1;
      } else if (deviceCount == 1) {
        //@@ WbLog is a provided logging API (similar to Log4J).
        //@@ The logging function wbLog takes a level which is either
        //@@ OFF, FATAL, ERROR, WARN, INFO, DEBUG, or TRACE and a
        //@@ message to be printed.
        wbLog(TRACE, "There is 1 device supporting CUDA");
      } else {
        wbLog(TRACE, "There are ", deviceCount,
              " devices supporting CUDA");
      }
    }

    wbLog(TRACE, "Device ", dev, " name: ", deviceProp.name);
    wbLog(TRACE, " Computational Capabilities: ", deviceProp.major, ".",
          deviceProp.minor);
    wbLog(TRACE, " Maximum global memory size: ",
          deviceProp.totalGlobalMem);
    wbLog(TRACE, " Maximum constant memory size: ",
          deviceProp.totalConstMem);
    wbLog(TRACE, " Maximum shared memory size per block: ",
          deviceProp.sharedMemPerBlock);
    wbLog(TRACE, " Maximum block dimensions: ",
          deviceProp.maxThreadsDim[0], " x ", deviceProp.maxThreadsDim[1],
          " x ", deviceProp.maxThreadsDim[2]);
    wbLog(TRACE, " Maximum grid dimensions: ", deviceProp.maxGridSize[0],
          " x ", deviceProp.maxGridSize[1], " x ",
          deviceProp.maxGridSize[2]);
    wbLog(TRACE, " Warp size: ", deviceProp.warpSize);
  }

  wbTime_stop(GPU, "Getting GPU Data."); //@@ stop the timer

  return 0;
}

Writing test.cu


In [ ]:
!nvcc test.cu -o test

In [ ]:
!./test

Trace main::30 There is 1 device supporting CUDA
Trace main::37 Device 0 name: NVIDIA A100-SXM4-40GB
Trace main::38  Computational Capabilities: 8.0
Trace main::40  Maximum global memory size: 42481549312
Trace main::42  Maximum constant memory size: 65536
Trace main::44  Maximum shared memory size per block: 49152
Trace main::46  Maximum block dimensions: 1024 x 1024 x 64
Trace main::49  Maximum grid dimensions: 2147483647 x 65535 x 65535
Trace main::52  Warp size: 32
[GPU    ] 0.000190976 Getting GPU Data.
